# Data Fusion Contest. Goodsification

# Часть 3

В этой части мы построим итоговую модель catboost с использованием ранее сгенерированных признаков. Некоторые идеи по параметрам модели взяты в открытом решении тут: https://github.com/exotol/data_fusion_vtb

In [3]:
import pandas as pd
import numpy as np
#import dask.dataframe as dd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import time
import gc
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tqdm.notebook import tqdm
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn import model_selection
import lightgbm as lgbm
import warnings
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import itertools


warnings.filterwarnings('ignore')

**Загрузка ранее сохраненных данных**

In [13]:
col_names=['receipt_id',  'receipt_dayofweek',  'receipt_time',  'item_name',  'item_quantity',  'item_price',  'item_nds_rate',  'category_id',  'brands',  'mean_item_quantity',  'sum_item_quantity',  'std_item_quantity',  'min_item_quantity',
 'max_item_quantity',  'mean_item_price',  'sum_item_price',  'std_item_price',  'min_item_price',  'max_item_price',  'mean_item_nds_rate',  'sum_item_nds_rate',  'std_item_nds_rate',  'min_item_nds_rate',
 'max_item_nds_rate',  'hours',  'mean_hours',  'sum_hours',  'std_hours',  'min_hours',  'max_hours',  'median_dayofweek',  'tokens',  0,  1,  2,  3,  4,  6,  7,  9,  11,  12,  13,  19,  20,  24,
 26,  27,  29,  30,  31,  35,  36,  37,  38,  39,  40,  41,  42,  43,  45,  46,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  60,  61,  62,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
 81,  82,  83,  84,  85,  90,  92,  96,  97,  100,  101,  102,  103,  105,  106,  107,  108,  109,  111,  114,  115,  117,  118,  120,  128,  130,  133,  138,  139,  140,  145,  150,  163,  164,  167,  177,  203,  204,
 'pred_category',  'first',  'last',  'second_last_word',  'second_first_word',  'first_word_len',  'last_word_len',  'last_first_word',  'receipt_type']


In [14]:
train=pd.read_csv('train_with_receipt_type.csv', names=col_names, header=0, index_col=0, encoding='utf-8-sig')

In [15]:
train.fillna('',inplace=True)

In [16]:
train

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands,mean_item_quantity,sum_item_quantity,std_item_quantity,min_item_quantity,max_item_quantity,mean_item_price,sum_item_price,std_item_price,min_item_price,max_item_price,mean_item_nds_rate,sum_item_nds_rate,std_item_nds_rate,min_item_nds_rate,max_item_nds_rate,hours,mean_hours,sum_hours,std_hours,min_hours,max_hours,median_dayofweek,tokens,0,1,2,3,4,6,7,9,...,96,97,100,101,102,103,105,106,107,108,109,111,114,115,117,118,120,128,130,133,138,139,140,145,150,163,164,167,177,203,204,pred_category,first,last,second_last_word,second_first_word,first_word_len,last_word_len,last_first_word,receipt_type
0,11,6,20:34,молоко 3 2 шт,2.000,8,2,78,,1.341835,5103.000,0.718658,1.000,12.000,7.853537,29867,0.675221,0,8,2.000000,7606,0.000000,2,2,20,15.450960,58760,3.495831,0,23,3,"['молоко', '3', '2', 'шт']",-2.944272,-1.102721,-1.650131,-1.420861,-1.526073,-1.512710,-1.203853,-1.186740,...,-1.444173,-1.402317,-1.261925,-1.362935,-1.406820,-1.567017,-1.187419,-1.415561,-1.336234,-1.101688,-1.273526,-1.318398,-1.312028,-1.527478,-1.362052,-1.212809,-1.520355,-1.298276,-2.457833,-1.338735,-1.708568,-1.555741,-1.645560,-2.074830,-1.773687,-1.155365,-1.396777,-1.243571,-1.283344,-1.923077,-0.823047,78,молоко,шт,2,3,6,2,штмолоко,7
1,39,4,11:28,компот из изюма 114 ккал,1.000,4,1,71,,1.201849,780.000,0.420460,1.000,4.000,4.000000,2596,0.000000,4,4,1.342065,871,1.258344,1,6,11,11.724191,7609,4.223616,0,23,4,"['компот', 'из', 'изюма', '114', 'ккал']",-2.419275,-1.695873,-1.432048,-1.563720,-1.657711,-1.406004,-1.310821,-1.178450,...,-1.833819,-1.295229,-1.086804,-1.289009,-1.153746,-2.012989,-1.602857,-1.867620,-1.743286,-1.427199,-2.020408,-1.712602,-2.628078,-1.442592,-2.499639,-1.664869,-2.172477,-1.492867,-2.565510,-1.395578,-1.673411,-2.133067,-1.959644,-2.214672,-1.566025,-1.643045,-1.323290,-1.651574,-1.772448,-1.544172,-1.205885,71,компот,ккал,114,из,6,4,ккалкомпот,6
2,39,4,11:28,макаронные изделия отварные масло сливочное 24...,1.000,4,1,71,,1.082397,1445.000,0.538570,1.000,18.000,4.000000,5340,0.000000,4,4,1.370787,1830,1.310625,1,6,11,11.788015,15737,4.518246,0,23,2,"['макаронные', 'изделия', 'отварные', 'масло',...",-3.362039,-1.379535,-1.464796,-1.602297,-1.863669,-1.550776,-1.374809,-1.202343,...,-1.680414,-1.461526,-1.328483,-1.314142,-1.213492,-2.122052,-2.082544,-1.739533,-1.935316,-1.371987,-2.517548,-1.738760,-2.141604,-1.760593,-2.412717,-1.879925,-2.641934,-1.459092,-2.643679,-1.536574,-1.806197,-2.667901,-2.414406,-2.793309,-1.888707,-1.696378,-1.740464,-1.952517,-2.075659,-1.803656,-1.737307,71,макаронные,ккал,240,изделия,10,4,ккалмакаронные,6
3,56,5,11:42,кофе капучино большой эден 18,1.000,12,1,70,,1.102445,947.000,0.322043,1.000,4.000,11.953434,10268,0.593415,0,12,1.000000,859,0.000000,1,1,11,12.299185,10565,4.941394,0,23,3,"['кофе', 'капучино', 'большой', 'эден', '18']",-2.330448,-1.291520,-1.654593,-1.712222,-2.178777,-1.314485,-1.358586,-1.270733,...,-1.136947,-1.378275,-1.156549,-1.481574,-1.548743,-1.619092,-1.627373,-1.273554,-1.493674,-1.851914,-1.951692,-1.592923,-2.355489,-1.293682,-2.214117,-1.850586,-1.729283,-1.562812,-1.772255,-1.919327,-1.420123,-1.710985,-2.198889,-1.955634,-1.440650,-1.763510,-1.503111,-1.571048,-1.288924,-1.550939,-1.720797,70,кофе,18,эден,капучино,4,2,18кофе,6
4,105,3,01:53,хлеб на сыворотке 350г,1.000,7,-1,84,,1.217787,1739.000,0.778688,1.000,10.000,6.600840,9426,0.489897,6,7,-0.980392,-1400,0.370090,-1,6,1,14.378151,20532,6.034693,0,23,3,"['хлеб', 'на', 'сыворотке', '350', 'г']",-2.153437,-1.589245,-1.491834,-1.632087,-1.960699,-1.233137,-1.390925,-1.512109,...,-1.396747,-1.254705,-1.403800,-1.510099,-1.583143,-1.454743,-1.375457,-1.255514,-1.823030,-1.699999,-1.672501,-1.290215,-1.601600,-1.858132,-1.361373,-1.639414,-1.734698,-1.529228,-2.064636,-1.728348,-1.389236,-2.258007,-1.860771,-1.668605,-1.616847,-1.430661,-1.468567,-1.704392,-1.892704,-1.770578,-1.994474,84,хлеб,350г,сыворот

**Валидация**

In [17]:
data_train, data_valid = model_selection.train_test_split(train,
                                                test_size=0.10, 
                                                stratify=train['category_id'], 
                                                shuffle=True, random_state=10)

In [21]:
feature_names = ['receipt_id','receipt_time', 'receipt_dayofweek', 'item_name', 'item_quantity', 'item_price', 'item_nds_rate', 'mean_item_quantity', 'sum_item_quantity', 'std_item_quantity', 
                 'min_item_quantity', 'max_item_quantity', 'mean_item_price', 'sum_item_price', 'std_item_price', 'min_item_price', 'max_item_price', 'mean_item_nds_rate', 'sum_item_nds_rate', 
                 'std_item_nds_rate', 'min_item_nds_rate', 'max_item_nds_rate', 'hours', 'mean_hours', 'sum_hours', 'std_hours', 'min_hours', 'max_hours', 'median_dayofweek',    
                0,  1,  2,  3,  4,  6,  7,  9,  11,  12,  13,  19,  20,  24,  26,  27,  29,  30,  31,  35,  36,  37,  38,  39,  40,  41,  42,  43,  45,  46,  49,  50,  51,  52,  53,  54,  55,
                56,  57,  58,  60,  61,  62,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  90,  92,  96,  97,  100,  101,  102,  103,  105,
                106,  107,  108,  109,  111,  114,  115,  117,  118,  120,  128,  130,  133,  138,  139,  140,  145,  150,  163,  164,  167,  177,  203,  
                'pred_category', 'first', 'last', 'first_word_len', 'last_word_len', 'second_last_word', 'second_first_word','last_first_word','receipt_type'] 

cat_features = ['receipt_id', 'receipt_time','receipt_dayofweek', 'hours', 'min_hours', 'max_hours', 'median_dayofweek', 'first', 'last', 'second_last_word', 'second_first_word',
                 'last_first_word','pred_category', 'receipt_type']

text_features = ['item_name']

target = 'category_id'                

In [25]:
model_catboost = CatBoostClassifier(
                cat_features=cat_features,
                text_features=text_features,
                border_count=254,
                bootstrap_type='Bernoulli',
                subsample=0.7,
                verbose=50,
                loss_function='MultiClass',
                eval_metric='TotalF1',
                early_stopping_rounds=100,
                task_type="GPU",
                iterations=2000,
                learning_rate=0.05,      
                one_hot_max_size=100, 
                depth=6, 
                l2_leaf_reg= 3,
                text_processing = {
        "tokenizers" : [{
            "tokenizer_id" : "Space",
            "separator_type" : "ByDelimiter",
            "delimiter" : " "
        }],

        "dictionaries" : [{
            "dictionary_id" : "BiGram",
            "token_level_type": "Letter",
            "max_dictionary_size" : "150000",
            "occurrence_lower_bound" : "1",
            "gram_order" : "2"
        },{
            "dictionary_id" : "Trigram",
            "max_dictionary_size" : "150000",
            "token_level_type": "Letter",
            "occurrence_lower_bound" : "1",
            "gram_order" : "3"
        },{
            "dictionary_id" : "Fourgram",
            "max_dictionary_size" : "150000",
            "token_level_type": "Letter",
            "occurrence_lower_bound" : "1",
            "gram_order" : "4"
        },{
            "dictionary_id" : "Fivegram",
            "max_dictionary_size" : "150000",
            "token_level_type": "Letter",
            "occurrence_lower_bound" : "1",
            "gram_order" : "5"
        },{
            "dictionary_id" : "Sixgram",
            "max_dictionary_size" : "150000",
            "token_level_type": "Letter",
            "occurrence_lower_bound" : "1",
            "gram_order" : "6"
        }
        ],

        "feature_processing" : {
            "default" : [
                    {
                    "dictionaries_names" : ["BiGram", "Trigram", "Fourgram", "Fivegram", "Sixgram"],
                    "feature_calcers" : ["BoW"],
                    "tokenizers_names" : ["Space"]
                },
                    {
                "dictionaries_names" : ["BiGram", "Trigram", "Fourgram", "Fivegram", "Sixgram"],
                "feature_calcers" : ["NaiveBayes"],
                "tokenizers_names" : ["Space"]
            },{
                "dictionaries_names" : [ "BiGram", "Trigram", "Fourgram", "Fivegram", "Sixgram"],
                "feature_calcers" : ["BM25"],
                "tokenizers_names" : ["Space"]
            },
            ],
        }
    }) 


In [26]:
model_catboost.fit(
                  data_train[feature_names], data_train[target],
                  eval_set=(data_valid[feature_names], data_valid[target]),
                  plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.1853942	test: 0.1737755	best: 0.1737755 (0)	total: 402ms	remaining: 13m 23s
50:	learn: 0.7121064	test: 0.7079944	best: 0.7079944 (50)	total: 17s	remaining: 10m 51s
100:	learn: 0.7993686	test: 0.8002720	best: 0.8002720 (100)	total: 32.7s	remaining: 10m 14s
150:	learn: 0.8215880	test: 0.8203402	best: 0.8204065 (146)	total: 48.3s	remaining: 9m 51s
200:	learn: 0.8338873	test: 0.8324572	best: 0.8326828 (197)	total: 1m 4s	remaining: 9m 34s
250:	learn: 0.8426193	test: 0.8411045	best: 0.8411045 (250)	total: 1m 19s	remaining: 9m 16s
300:	learn: 0.8515240	test: 0.8484298	best: 0.8484298 (300)	total: 1m 35s	remaining: 8m 59s
350:	learn: 0.8543664	test: 0.8519722	best: 0.8519722 (350)	total: 1m 51s	remaining: 8m 45s
400:	learn: 0.8573695	test: 0.8566826	best: 0.8566826 (400)	total: 2m 8s	remaining: 8m 31s
450:	learn: 0.8603085	test: 0.8593768	best: 0.8597649 (441)	total: 2m 25s	remaining: 8m 18s
500:	learn: 0.8628602	test: 0.8608911	best: 0.8612105 (475)	total: 2m 42s	remaining: 8m 5s


**Посмотрим на важность признаков:**

In [27]:
importance = model_catboost.get_feature_importance(prettified=True)
importance.head(50)

,Feature Id,Importances
0,pred_category,29.880895
1,item_name,15.691363
2,receipt_type,15.087613
3,first,4.713306
4,second_first_word,3.270989
5,last_first_word,2.341115
6,71,2.295668
7,last,2.178442
8,84,1.994487
9,78,1.857928


**Сохраняем модель:**

In [29]:
pool = Pool(
    data=train[feature_names], 
    label=train[target],
    
feature_names = ['receipt_id','receipt_time', 'receipt_dayofweek', 'item_name', 'item_quantity', 'item_price', 'item_nds_rate', 'mean_item_quantity', 'sum_item_quantity', 'std_item_quantity', 
                 'min_item_quantity', 'max_item_quantity', 'mean_item_price', 'sum_item_price', 'std_item_price', 'min_item_price', 'max_item_price', 'mean_item_nds_rate', 'sum_item_nds_rate', 
                 'std_item_nds_rate', 'min_item_nds_rate', 'max_item_nds_rate', 'hours', 'mean_hours', 'sum_hours', 'std_hours', 'min_hours', 'max_hours', 'median_dayofweek',    
                0,  1,  2,  3,  4,  6,  7,  9,  11,  12,  13,  19,  20,  24,  26,  27,  29,  30,  31,  35,  36,  37,  38,  39,  40,  41,  42,  43,  45,  46,  49,  50,  51,  52,  53,  54,  55,
                56,  57,  58,  60,  61,  62,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  90,  92,  96,  97,  100,  101,  102,  103,  105,
                106,  107,  108,  109,  111,  114,  115,  117,  118,  120,  128,  130,  133,  138,  139,  140,  145,  150,  163,  164,  167,  177,  203,  
                'pred_category', 'first','last', 

                
                'first_word_len', 'last_word_len',
                 'second_last_word', 'second_first_word',
                 
                 'last_first_word',
                 'receipt_type'
                ], 

cat_features = ['receipt_id', 'receipt_time','receipt_dayofweek', 'hours', 'min_hours', 'max_hours', 'median_dayofweek', 'first', 'last', 'second_last_word', 'second_first_word',
                 'last_first_word','pred_category', 'receipt_type'
                ],

text_features = ['item_name']

)

In [31]:
model_catboost.save_model( "/content/drive/MyDrive/Data Fusion Contest VTB/catboost/model_catboost.cbm", format="cbm", pool=pool)

Данное решение на публичном лидерборде показало скор 0.8562302, что позволило войти в ТОП-20 публичного лидерборда.
